In [31]:
import os
import yaml
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from typing import List, TypedDict, Annotated
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
"""
from langchain.vectorstores import FAISS
from langchain_core.graph import Graph, Node, StartNode, StopNode
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
"""

# Load environment variables
load_dotenv()
api_key = os.environ["groq"]

# Initialize LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)


In [32]:
llm.invoke("hi").content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [33]:
llm.invoke("where is taj mahal?").content

'The Taj Mahal is located in Agra, a city in the Indian state of Uttar Pradesh. It is situated on the southern bank of the Yamuna River, approximately 230 kilometers (143 miles) south of New Delhi, the capital city of India.\n\nThe exact address of the Taj Mahal is:\n\nTaj Mahal, Dharmapuri, Forest Colony, Tajganj, Agra, Uttar Pradesh 282001, India\n\nIt is one of the most famous and iconic monuments in the world, and a popular tourist destination, attracting millions of visitors each year.'

In [ ]:
resume=""
with open('data/job_description.txt', 'r') as file:
    job_description=file.read()
loader = PyPDFLoader("data/resume_with_capg.pdf")
documents = loader.load()
for i in documents:
    resume+=(i.page_content.strip())
    
with open('prompts.yaml', 'r') as file:
    data = yaml.safe_load(file)
    
def intro():

    """Choose a key from the following list to use the corresponding prompt template:"""
    print("*"*100)
    for i,j in enumerate(data.keys()):
        print(f"{i+1}. {j}")
    i=input("Enter the number corresponding to the key you want to use: ")
    print("selected number is ",i)
    key=list(data.keys())[int(i)-1]
    return key

def func(key):
    prompt_template = PromptTemplate(
        input_variables=["resume", "job_description"],
        template=data[key]["prompt"]
    )
    chain=prompt_template | llm
    response=chain.invoke(data[key]["inputs"]).content
    return response

In [ ]:
"""
creating a tool for llm that takes user input and uses the func function to get the response


"""

@tool
def get_response(resume: str, job_description: str) -> str: 
    """
    Use this tool to get a response based on the resume and job description.
    """
    key=intro()
    response = func(key)
    return response


In [51]:
r

"Dinesh\tSagar\n+91-8121909150\n\t\nMail\n\t\nGitHub\n\t\nLinkedIn\tProfile\nPassionate\tData\tScientist\twith\ta\tdrive\tto\ttackle\treal-world\tchallenges\tthrough\tdata\tanalysis\tand\tmachine\tlearning.\nCommitted\tto\textracting\tvalue\tfrom\tcomplex\tdata\tand\tdelivering\tactionable\tinsights\tto\tinform\tbusiness\tdecisions\t\nand\ndrive\tgrowth.\tEager\tto\tcontinuously\tlearn\tand\timprove\tto\tdeliver\timpactful\tsolutions.\nExperience\nCapgemini\nConsultant\nFEB\t2025-\tPRESENT\nAmazon\tDevelopment\tCentre\tIndia\nQuality\tSpecialist\nJAN\t2020\t-\tDEC\t2024\nImplemented\tan\toutlier\tdetection\tprogram\tto\nenhance\taddress\tdata\tquality,\timproving\tlogistics\nperformance\tfor\tAmazon\tLogistics.\nAnalyzed\tlarge-scale\tdatasets\tusing\tstatistical\nmethods\tto\toptimize\tdelivery\toperations\tand\nresource\tplanning.\nBuilt\ttime\tseries\tmodels\tto\tforecast\tpeak\torder\nvolumes,\tenabling\tproactive\tlogistics\nmanagement.\nDeveloped\tmachine\tlearning\tmodels\tto\te